In [1]:
# Hipotesis
## 1. Los tiburones atacan porque se ven amenazados o los humanos invaden su espacio.
## Evolución por años del tipo de ataques, si son no provocados o injerencia humana.
## 2. 10 personas al año mueren por tiburones, hacer media al año de muertes. 
## 3. Ha aumentado en algún país? COmprobar
## 3. Dónde se producen más ataques de tiburones. Habría que comprobar outliers en fechas porque es probable que 
## en la década de los 20, por ejemplo se tengan menos datos.
## Futuras: Saber si los que más han atacado son las especies más afectadas por la caza o algo así.

In [96]:
import pandas as pd
import numpy as np
import matplotlib as plt
import re

In [97]:
data = pd.read_csv('../your-code/dataset_origin.csv', engine='python')
display(data)

#Importación del documento

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
0,2016.09.18.c,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,...,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.c-NSB.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.c,2016.09.18.c,5993,NaN,NaN
1,2016.09.18.b,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,Chucky Luciano,M,...,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.b-Luciano.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.b,2016.09.18.b,5992,NaN,NaN
2,2016.09.18.a,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,...,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.a-NSB.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.a,2016.09.18.a,5991,NaN,NaN
3,2016.09.17,17-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Thirteenth Beach,Surfing,Rory Angiolella,M,...,NaN,"The Age, 9/18/2016",2016.09.17-Angiolella.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.17,2016.09.17,5990,NaN,NaN
4,2016.09.15,16-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Bells Beach,Surfing,male,M,...,2 m shark,"The Age, 9/16/2016",2016.09.16-BellsBeach.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.16,2016.09.15,5989,NaN,NaN
5,2016.09.15.R,15-Sep-16,2016,Boat,AUSTRALIA,Western Australia,Bunbury,Fishing,Occupant: Ben Stratton,NaN,...,NaN,"West Australian, 9/15/2016",2016.09.15.R-boat.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.15.R,2016.09.15.R,5988,NaN,NaN
6,2016.09.11,11-Sep-16,2016,Unprovoked,USA,Florida,"Ponte Vedra, St. Johns County",Wading,male,M,...,3' to 4' shark,"News4Jax, 9/11/2016",2016.09.11-PonteVedra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.11,2016.09.11,5987,NaN,NaN
7,2016.09.07,07-Sep-16,2016,Unprovoked,USA,Hawaii,"Makaha, Oahu",Swimming,female,F,...,"Tiger shark, 10?","Hawaii News Now, 9/7/2016",2016.09.07-Oahu.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.07,2016.09.07,5986,NaN,NaN
8,2016.09.06,06-Sep-16,2016,Unprovoked,NEW CALEDONIA,North Province,Koumac,Kite surfing,David Jewell,M,...,NaN,"TVANouvelles, 9/6/2016",2016.09.06-Jewell.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.06,2016.09.06,5985,NaN,NaN
9,2016.09.05.b,05-Sep-16,2016,Unprovoked,USA,South Carolina,"Kingston Plantation, Myrtle Beach, Horry County",Boogie boarding,Rylie Williams,F,...,NaN,"C. Creswell, GSAF",2016.09.05.b-Williams.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.05.b,2016.09.05.b,5984,NaN,NaN


In [4]:
x = data[(data['Country'] =='OCEAN')]
display(x) #Comprobación del valor Ocean en Country

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
5503,1887.00.00,1887,1887,Sea Disaster,OCEAN,"Somewhere between Philadelphia and Hiogo, Japan",NaN,British ship Macedon was thrown on her beam en...,a ship's steward,M,...,NaN,"Galveston Daily News, 2/11/1888",1887.00.00-Macedom.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1887.00.00,1887.00.00,490,NaN,NaN


In [98]:
display(data['Country'].value_counts()) #Cuento el tipo de valores de Country para ver cómo unificarlo

USA                             2116
AUSTRALIA                       1279
SOUTH AFRICA                     565
PAPUA NEW GUINEA                 133
NEW ZEALAND                      125
BRAZIL                           102
BAHAMAS                           98
MEXICO                            81
ITALY                             71
FIJI                              62
PHILIPPINES                       59
REUNION                           57
NEW CALEDONIA                     51
MOZAMBIQUE                        44
CUBA                              42
SPAIN                             40
INDIA                             37
EGYPT                             36
CROATIA                           34
PANAMA                            32
JAPAN                             32
SOLOMON ISLANDS                   29
IRAN                              29
GREECE                            25
HONG KONG                         24
JAMAICA                           23
FRENCH POLYNESIA                  22
I

In [99]:
data.columns
x = data[(data["Location"].isnull() == True)]
display(x) #Quiero ver las localizaciones que tienen Nan para probablemente eliminarlas

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
21,2016.07.28.R,28-Jul-16,2016,Unprovoked,CHINA,Hong Kong,NaN,Swimming,Justus Franz,M,...,NaN,"Klassick, 7/28/2016",2016.07.28.R-Franz.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.07.28.R,2016.07.28.R,5972,NaN,NaN
42,2016.06.24,24-Jun-16,2016,Unprovoked,COLUMBIA,Isla Provedencia,NaN,Scuba Diving,Arturo Velez,M,...,"Caribbean reef shark, 4.5'",Dr. A. Velez,2016.06.24-Velez.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.06.24,2016.06.24,5951,NaN,NaN
73,2016.04.08,08-Apr-16,2016,Invalid,CAPE VERDE,Boa Vista Island,NaN,NaN,a British citizen,M,...,Shark involvement not confirmed,L.O.Guttke,2016.04.08-CapeVerde.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.04.08,2016.04.08,5920,NaN,NaN
80,2016.03.26,26-Mar-16,2016,Provoked,BAHAMAS,NaN,NaN,NaN,Henry Kreckman,M,...,"Nurse shark, 2.5-ft","Wisconsin State Journal, 4/2/2016",2016.03.26-Kreckman.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.03.26,2016.03.26,5913,NaN,NaN
83,2016.03.10,10-Mar-16,2016,Unprovoked,Fiji,Vanua Levu,NaN,Diving for beche-de-mer,Maika Tabua,M,...,NaN,"Fiji Sun, 3/12/2016",2016.03.10-Tabua.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.03.10,2016.03.10,5910,NaN,NaN
94,2016.01.29,29-Jan-16,2016,Boat,SOUTH AFRICA,KwaZulu-Natal,NaN,Kayak fishing,Dev De Lange,M,...,NaN,"Nine News, 2/1/2016",2016.01.29-DeLange.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.01.29,2016.01.29,5899,NaN,NaN
151,2015.09.00,Sep-15,2015,Unprovoked,FIJI,NaN,NaN,Spearfishing,Viliame Lautiki,M,...,Tiger shark,"Fiji Times, 2/8/2016",2015.09.00-Lautiki.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2015.09.00,2015.09.00,5841,NaN,NaN
171,2015.07.06,06-Jul-15,2015,Invalid,FRENCH POLYNESIA,Bora Bora,NaN,Swimming,Joe Termini,M,...,No shark involvement,"Hollywood Life, 7/6/2015",2015.07.06-Termini.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2015.07.06,2015.07.06,5821,NaN,NaN
217,2015.03.29,29-Mar-15,2015,Invalid,ITALY,Sardinia,NaN,Diving,Eugenio Masala,M,...,Shark involvement not cofirmed,"A. de Maddalena, GSAF",2015.03.29-Masala.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2015.03.29,2015.03.29,5775,NaN,NaN
225,2015.02.15,15-Feb-15,2015,Boat,ATLANTIC OCEAN,NaN,NaN,Transatlantic Rowing,"Avalon, a carbon kevlar monohull: 8 occupants",NaN,...,White shark,"Yorkshire Post, 3/16/2014",2015.02.15-Avalon.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2015.02.15,2015.02.15,5767,NaN,NaN


In [7]:
data.columns #Saber el numero de columnas 

Index(['Case Number', 'Date', 'Year', 'Type', 'Country', 'Area', 'Location',
       'Activity', 'Name', 'Sex ', 'Age', 'Injury', 'Fatal (Y/N)', 'Time',
       'Species ', 'Investigator or Source', 'pdf', 'href formula', 'href',
       'Case Number.1', 'Case Number.2', 'original order', 'Unnamed: 22',
       'Unnamed: 23'],
      dtype='object')

In [8]:
data = data.transpose() #Hago transpose para ver mejor el dataframe

In [17]:
unnamed = data[(data["Unnamed: 23"].isnull() == True)]
unnamed.shape #Compruebo que Unnamed:23 tiene como preveía muchos 

(5990, 24)

In [18]:
unnamed2 = data[(data["Unnamed: 22"].isnull() == True)]
unnamed2.shape #Compruebo que Unnamed:22 tiene como preveía muchos Nan

(5991, 24)

In [19]:
casNumber1 = data[(data["Case Number.1"].isnull() == True)]
casNumber1.shape
x = data["Case Number.1"]== data["Case Number"] #Aquí compruebo que ambas columnas contienen en la mayor 
#parte de los casos los mismos valores. Justifico eliminarlos
x.value_counts()

True     5979
False      13
dtype: int64

In [ ]:
casNumber1 = data[(data["Case Number.2"].isnull() == True)]
casNumber1.shape
y = data["Case Number.2"]== data["Case Number"]
y.value_counts() #Aquí compruebo que ambas columnas contienen en la mayor 
#parte de los casos los mismos valores. Justifico eliminarlos

In [100]:
def replaceString(e):
    e=str(e)
    e = re.findall('\d\d\d\.(\d\d).', e)
    if e ==[]:
        return 0
    return e[0]

In [101]:
data['Case Number'] = data['Case Number'].apply(replaceString)
display(data['Case Number'])

0       09
1       09
2       09
3       09
4       09
5       09
6       09
7       09
8       09
9       09
10      09
11      09
12      09
13      08
14      08
15      08
16      08
17      08
18      08
19      08
20      07
21      07
22      07
23      07
24      07
25      07
26      07
27      07
28      07
29      07
        ..
5962     0
5963     0
5964     0
5965     0
5966     0
5967     0
5968     0
5969     0
5970     0
5971     0
5972     0
5973     0
5974     0
5975     0
5976     0
5977     0
5978     0
5979     0
5980     0
5981     0
5982     0
5983     0
5984     0
5985     0
5986     0
5987     0
5988     0
5989     0
5990     0
5991     0
Name: Case Number, Length: 5992, dtype: object

In [102]:
data = data[(data['Case Number']!='00') & (data['Case Number']!=0)]
display(data)

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
0,09,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,...,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.c-NSB.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.c,2016.09.18.c,5993,NaN,NaN
1,09,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,Chucky Luciano,M,...,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.b-Luciano.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.b,2016.09.18.b,5992,NaN,NaN
2,09,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,...,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.a-NSB.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.a,2016.09.18.a,5991,NaN,NaN
3,09,17-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Thirteenth Beach,Surfing,Rory Angiolella,M,...,NaN,"The Age, 9/18/2016",2016.09.17-Angiolella.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.17,2016.09.17,5990,NaN,NaN
4,09,16-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Bells Beach,Surfing,male,M,...,2 m shark,"The Age, 9/16/2016",2016.09.16-BellsBeach.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.16,2016.09.15,5989,NaN,NaN
5,09,15-Sep-16,2016,Boat,AUSTRALIA,Western Australia,Bunbury,Fishing,Occupant: Ben Stratton,NaN,...,NaN,"West Australian, 9/15/2016",2016.09.15.R-boat.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.15.R,2016.09.15.R,5988,NaN,NaN
6,09,11-Sep-16,2016,Unprovoked,USA,Florida,"Ponte Vedra, St. Johns County",Wading,male,M,...,3' to 4' shark,"News4Jax, 9/11/2016",2016.09.11-PonteVedra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.11,2016.09.11,5987,NaN,NaN
7,09,07-Sep-16,2016,Unprovoked,USA,Hawaii,"Makaha, Oahu",Swimming,female,F,...,"Tiger shark, 10?","Hawaii News Now, 9/7/2016",2016.09.07-Oahu.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.07,2016.09.07,5986,NaN,NaN
8,09,06-Sep-16,2016,Unprovoked,NEW CALEDONIA,North Province,Koumac,Kite surfing,David Jewell,M,...,NaN,"TVANouvelles, 9/6/2016",2016.09.06-Jewell.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.06,2016.09.06,5985,NaN,NaN
9,09,05-Sep-16,2016,Unprovoked,USA,South Carolina,"Kingston Plantation, Myrtle Beach, Horry County",Boogie boarding,Rylie Williams,F,...,NaN,"C. Creswell, GSAF",2016.09.05.b-Williams.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.05.b,2016.09.05.b,5984,NaN,NaN


In [103]:
data = data.rename(index=str, columns={"Case Number": "Month"})
display(data)

,Month,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
0,09,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,...,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.c-NSB.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.c,2016.09.18.c,5993,NaN,NaN
1,09,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,Chucky Luciano,M,...,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.b-Luciano.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.b,2016.09.18.b,5992,NaN,NaN
2,09,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,...,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.a-NSB.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.a,2016.09.18.a,5991,NaN,NaN
3,09,17-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Thirteenth Beach,Surfing,Rory Angiolella,M,...,NaN,"The Age, 9/18/2016",2016.09.17-Angiolella.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.17,2016.09.17,5990,NaN,NaN
4,09,16-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Bells Beach,Surfing,male,M,...,2 m shark,"The Age, 9/16/2016",2016.09.16-BellsBeach.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.16,2016.09.15,5989,NaN,NaN
5,09,15-Sep-16,2016,Boat,AUSTRALIA,Western Australia,Bunbury,Fishing,Occupant: Ben Stratton,NaN,...,NaN,"West Australian, 9/15/2016",2016.09.15.R-boat.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.15.R,2016.09.15.R,5988,NaN,NaN
6,09,11-Sep-16,2016,Unprovoked,USA,Florida,"Ponte Vedra, St. Johns County",Wading,male,M,...,3' to 4' shark,"News4Jax, 9/11/2016",2016.09.11-PonteVedra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.11,2016.09.11,5987,NaN,NaN
7,09,07-Sep-16,2016,Unprovoked,USA,Hawaii,"Makaha, Oahu",Swimming,female,F,...,"Tiger shark, 10?","Hawaii News Now, 9/7/2016",2016.09.07-Oahu.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.07,2016.09.07,5986,NaN,NaN
8,09,06-Sep-16,2016,Unprovoked,NEW CALEDONIA,North Province,Koumac,Kite surfing,David Jewell,M,...,NaN,"TVANouvelles, 9/6/2016",2016.09.06-Jewell.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.06,2016.09.06,5985,NaN,NaN
9,09,05-Sep-16,2016,Unprovoked,USA,South Carolina,"Kingston Plantation, Myrtle Beach, Horry County",Boogie boarding,Rylie Williams,F,...,NaN,"C. Creswell, GSAF",2016.09.05.b-Williams.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.05.b,2016.09.05.b,5984,NaN,NaN


In [104]:
data = data[(data['Year']>=1956)]
display(data) #Voy a analizar la evolución en los últimos 60 años.

,Month,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
0,09,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,...,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.c-NSB.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.c,2016.09.18.c,5993,NaN,NaN
1,09,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,Chucky Luciano,M,...,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.b-Luciano.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.b,2016.09.18.b,5992,NaN,NaN
2,09,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,...,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.a-NSB.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.a,2016.09.18.a,5991,NaN,NaN
3,09,17-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Thirteenth Beach,Surfing,Rory Angiolella,M,...,NaN,"The Age, 9/18/2016",2016.09.17-Angiolella.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.17,2016.09.17,5990,NaN,NaN
4,09,16-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Bells Beach,Surfing,male,M,...,2 m shark,"The Age, 9/16/2016",2016.09.16-BellsBeach.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.16,2016.09.15,5989,NaN,NaN
5,09,15-Sep-16,2016,Boat,AUSTRALIA,Western Australia,Bunbury,Fishing,Occupant: Ben Stratton,NaN,...,NaN,"West Australian, 9/15/2016",2016.09.15.R-boat.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.15.R,2016.09.15.R,5988,NaN,NaN
6,09,11-Sep-16,2016,Unprovoked,USA,Florida,"Ponte Vedra, St. Johns County",Wading,male,M,...,3' to 4' shark,"News4Jax, 9/11/2016",2016.09.11-PonteVedra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.11,2016.09.11,5987,NaN,NaN
7,09,07-Sep-16,2016,Unprovoked,USA,Hawaii,"Makaha, Oahu",Swimming,female,F,...,"Tiger shark, 10?","Hawaii News Now, 9/7/2016",2016.09.07-Oahu.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.07,2016.09.07,5986,NaN,NaN
8,09,06-Sep-16,2016,Unprovoked,NEW CALEDONIA,North Province,Koumac,Kite surfing,David Jewell,M,...,NaN,"TVANouvelles, 9/6/2016",2016.09.06-Jewell.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.06,2016.09.06,5985,NaN,NaN
9,09,05-Sep-16,2016,Unprovoked,USA,South Carolina,"Kingston Plantation, Myrtle Beach, Horry County",Boogie boarding,Rylie Williams,F,...,NaN,"C. Creswell, GSAF",2016.09.05.b-Williams.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.05.b,2016.09.05.b,5984,NaN,NaN


In [105]:
data['Type'].value_counts() 

Unprovoked      2938
Provoked         374
Invalid          327
Boat             184
Sea Disaster      84
Boating            3
Name: Type, dtype: int64

In [106]:
data = data.replace(to_replace='Boating', value='Boat', regex=True)

In [27]:
data['Type'].value_counts() 

Unprovoked      2938
Provoked         374
Invalid          327
Boat             187
Sea Disaster      84
Name: Type, dtype: int64

In [28]:
data['Country'].value_counts()

USA                               1698
AUSTRALIA                          682
SOUTH AFRICA                       431
BRAZIL                              95
BAHAMAS                             82
NEW ZEALAND                         77
PAPUA NEW GUINEA                    75
MEXICO                              53
REUNION                             51
NEW CALEDONIA                       43
MOZAMBIQUE                          36
FIJI                                35
ITALY                               29
PHILIPPINES                         29
EGYPT                               25
JAPAN                               24
HONG KONG                           20
FRENCH POLYNESIA                    17
SPAIN                               16
SOLOMON ISLANDS                     14
PANAMA                              11
VIETNAM                             10
TAIWAN                               9
MARSHALL ISLANDS                     9
TONGA                                9
BERMUDA                  

In [107]:
data = data[data['Country'].isin(['USA', 'AUSTRALIA', 'SOUTH AFRICA', 'RAZIL','BAHAMAS','NEW ZEALAND', 'PAPUA NEW GUINEA', 'MEXICO', 'REUNION','NEW CALEDONIA','MOZAMBIQUE','FIJI ITALY','PHILIPPINES', 'EGYPT', 'JAPAN', 'HONG KONG','FRENCH', 'POLYNESIA', 'SPAIN','SOLOMON ISLANDS', 'PANAMA', 'VIETNAM'])]

In [30]:
display(data)

,Month,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
0,09,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,...,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.c-NSB.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.c,2016.09.18.c,5993,NaN,NaN
1,09,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,Chucky Luciano,M,...,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.b-Luciano.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.b,2016.09.18.b,5992,NaN,NaN
2,09,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,...,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.a-NSB.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.a,2016.09.18.a,5991,NaN,NaN
3,09,17-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Thirteenth Beach,Surfing,Rory Angiolella,M,...,NaN,"The Age, 9/18/2016",2016.09.17-Angiolella.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.17,2016.09.17,5990,NaN,NaN
4,09,16-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Bells Beach,Surfing,male,M,...,2 m shark,"The Age, 9/16/2016",2016.09.16-BellsBeach.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.16,2016.09.15,5989,NaN,NaN
5,09,15-Sep-16,2016,Boat,AUSTRALIA,Western Australia,Bunbury,Fishing,Occupant: Ben Stratton,NaN,...,NaN,"West Australian, 9/15/2016",2016.09.15.R-boat.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.15.R,2016.09.15.R,5988,NaN,NaN
6,09,11-Sep-16,2016,Unprovoked,USA,Florida,"Ponte Vedra, St. Johns County",Wading,male,M,...,3' to 4' shark,"News4Jax, 9/11/2016",2016.09.11-PonteVedra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.11,2016.09.11,5987,NaN,NaN
7,09,07-Sep-16,2016,Unprovoked,USA,Hawaii,"Makaha, Oahu",Swimming,female,F,...,"Tiger shark, 10?","Hawaii News Now, 9/7/2016",2016.09.07-Oahu.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.07,2016.09.07,5986,NaN,NaN
8,09,06-Sep-16,2016,Unprovoked,NEW CALEDONIA,North Province,Koumac,Kite surfing,David Jewell,M,...,NaN,"TVANouvelles, 9/6/2016",2016.09.06-Jewell.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.06,2016.09.06,5985,NaN,NaN
9,09,05-Sep-16,2016,Unprovoked,USA,South Carolina,"Kingston Plantation, Myrtle Beach, Horry County",Boogie boarding,Rylie Williams,F,...,NaN,"C. Creswell, GSAF",2016.09.05.b-Williams.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.05.b,2016.09.05.b,5984,NaN,NaN


In [108]:
data = data[(data["Country"].isnull() != True)]
display(data)

,Month,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
0,09,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,...,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.c-NSB.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.c,2016.09.18.c,5993,NaN,NaN
1,09,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,Chucky Luciano,M,...,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.b-Luciano.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.b,2016.09.18.b,5992,NaN,NaN
2,09,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,...,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.a-NSB.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.a,2016.09.18.a,5991,NaN,NaN
3,09,17-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Thirteenth Beach,Surfing,Rory Angiolella,M,...,NaN,"The Age, 9/18/2016",2016.09.17-Angiolella.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.17,2016.09.17,5990,NaN,NaN
4,09,16-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Bells Beach,Surfing,male,M,...,2 m shark,"The Age, 9/16/2016",2016.09.16-BellsBeach.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.16,2016.09.15,5989,NaN,NaN
5,09,15-Sep-16,2016,Boat,AUSTRALIA,Western Australia,Bunbury,Fishing,Occupant: Ben Stratton,NaN,...,NaN,"West Australian, 9/15/2016",2016.09.15.R-boat.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.15.R,2016.09.15.R,5988,NaN,NaN
6,09,11-Sep-16,2016,Unprovoked,USA,Florida,"Ponte Vedra, St. Johns County",Wading,male,M,...,3' to 4' shark,"News4Jax, 9/11/2016",2016.09.11-PonteVedra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.11,2016.09.11,5987,NaN,NaN
7,09,07-Sep-16,2016,Unprovoked,USA,Hawaii,"Makaha, Oahu",Swimming,female,F,...,"Tiger shark, 10?","Hawaii News Now, 9/7/2016",2016.09.07-Oahu.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.07,2016.09.07,5986,NaN,NaN
8,09,06-Sep-16,2016,Unprovoked,NEW CALEDONIA,North Province,Koumac,Kite surfing,David Jewell,M,...,NaN,"TVANouvelles, 9/6/2016",2016.09.06-Jewell.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.06,2016.09.06,5985,NaN,NaN
9,09,05-Sep-16,2016,Unprovoked,USA,South Carolina,"Kingston Plantation, Myrtle Beach, Horry County",Boogie boarding,Rylie Williams,F,...,NaN,"C. Creswell, GSAF",2016.09.05.b-Williams.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.05.b,2016.09.05.b,5984,NaN,NaN


In [32]:
data['Country'].value_counts()

USA                 1698
AUSTRALIA            682
SOUTH AFRICA         431
BAHAMAS               82
NEW ZEALAND           77
PAPUA NEW GUINEA      75
MEXICO                53
REUNION               51
NEW CALEDONIA         43
MOZAMBIQUE            36
PHILIPPINES           29
EGYPT                 25
JAPAN                 24
HONG KONG             20
SPAIN                 16
SOLOMON ISLANDS       14
PANAMA                11
VIETNAM               10
Name: Country, dtype: int64

In [ ]:
spain = data[(data['Country']=='SPAIN')]
display(spain)

In [ ]:
data.iloc[3300:3500,[0,2,4,5]]

In [109]:
cust_counts = data['Area'].value_counts()
cust_list = cust_counts[cust_counts >= 5].index.tolist()
data = data[data['Area'].isin(cust_list)]

display(data)

,Month,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
0,09,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,...,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.c-NSB.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.c,2016.09.18.c,5993,NaN,NaN
1,09,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,Chucky Luciano,M,...,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.b-Luciano.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.b,2016.09.18.b,5992,NaN,NaN
2,09,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,...,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.a-NSB.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.a,2016.09.18.a,5991,NaN,NaN
3,09,17-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Thirteenth Beach,Surfing,Rory Angiolella,M,...,NaN,"The Age, 9/18/2016",2016.09.17-Angiolella.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.17,2016.09.17,5990,NaN,NaN
4,09,16-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Bells Beach,Surfing,male,M,...,2 m shark,"The Age, 9/16/2016",2016.09.16-BellsBeach.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.16,2016.09.15,5989,NaN,NaN
5,09,15-Sep-16,2016,Boat,AUSTRALIA,Western Australia,Bunbury,Fishing,Occupant: Ben Stratton,NaN,...,NaN,"West Australian, 9/15/2016",2016.09.15.R-boat.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.15.R,2016.09.15.R,5988,NaN,NaN
6,09,11-Sep-16,2016,Unprovoked,USA,Florida,"Ponte Vedra, St. Johns County",Wading,male,M,...,3' to 4' shark,"News4Jax, 9/11/2016",2016.09.11-PonteVedra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.11,2016.09.11,5987,NaN,NaN
7,09,07-Sep-16,2016,Unprovoked,USA,Hawaii,"Makaha, Oahu",Swimming,female,F,...,"Tiger shark, 10?","Hawaii News Now, 9/7/2016",2016.09.07-Oahu.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.07,2016.09.07,5986,NaN,NaN
8,09,06-Sep-16,2016,Unprovoked,NEW CALEDONIA,North Province,Koumac,Kite surfing,David Jewell,M,...,NaN,"TVANouvelles, 9/6/2016",2016.09.06-Jewell.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.06,2016.09.06,5985,NaN,NaN
9,09,05-Sep-16,2016,Unprovoked,USA,South Carolina,"Kingston Plantation, Myrtle Beach, Horry County",Boogie boarding,Rylie Williams,F,...,NaN,"C. Creswell, GSAF",2016.09.05.b-Williams.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.05.b,2016.09.05.b,5984,NaN,NaN


In [ ]:
data['Area'].value_counts()

In [110]:
def newString(e):
    e=str(e)
    if (re.search('California', e)):
        return 'California'
    if (re.search('Florida', e)):
        return 'Florida'
    if (re.search('Bay ', e)):
        return 'Milne Bay'
    else:
        return e

In [111]:
k = data['Area'] = data['Area'].apply(newString)
display(k)

0                     Florida
1                     Florida
2                     Florida
3                    Victoria
4                    Victoria
5           Western Australia
6                     Florida
7                      Hawaii
8              North Province
9              South Carolina
10          Western Australia
11                    Florida
12                 California
13                    Florida
14                    Florida
16                    Florida
18                     Hawaii
19                    Florida
22          Western Australia
23                    Florida
24            New South Wales
26                   Tasmania
27              Abaco Islands
28                 Queensland
29                    Alabama
30                    Florida
31                    Florida
32             South Carolina
35                 California
36              Massachusetts
                ...          
3983                  Florida
3984            KwaZulu-Natal
3996      

In [ ]:
display(data)

In [112]:
data_clean = data.drop(["Unnamed: 22", "Unnamed: 23", "Case Number.2",'Time','original order','Injury', "Case Number.1",'Location', 'Investigator or Source', 'pdf', 'href formula', 'href', 'Name', 'Date'], axis=1)
display(data_clean)

,Month,Year,Type,Country,Area,Activity,Sex,Age,Fatal (Y/N),Species
0,09,2016,Unprovoked,USA,Florida,Surfing,M,16,N,NaN
1,09,2016,Unprovoked,USA,Florida,Surfing,M,36,N,NaN
2,09,2016,Unprovoked,USA,Florida,Surfing,M,43,N,NaN
3,09,2016,Unprovoked,AUSTRALIA,Victoria,Surfing,M,NaN,N,NaN
4,09,2016,Unprovoked,AUSTRALIA,Victoria,Surfing,M,NaN,N,2 m shark
5,09,2016,Boat,AUSTRALIA,Western Australia,Fishing,NaN,NaN,N,NaN
6,09,2016,Unprovoked,USA,Florida,Wading,M,60s,N,3' to 4' shark
7,09,2016,Unprovoked,USA,Hawaii,Swimming,F,51,N,"Tiger shark, 10?"
8,09,2016,Unprovoked,NEW CALEDONIA,North Province,Kite surfing,M,50,Y,NaN
9,09,2016,Unprovoked,USA,South Carolina,Boogie boarding,F,12,N,NaN


In [37]:
data_clean['Fatal (Y/N)'].value_counts()

N          2643
Y           365
UNKNOWN      30
 N            7
Name: Fatal (Y/N), dtype: int64

In [113]:
data_clean = data_clean[(data_clean['Fatal (Y/N)']=='Y') | (data_clean['Fatal (Y/N)']=='N') | (data_clean['Fatal (Y/N)']=='UNKNOWN')]
display(data_clean)

,Month,Year,Type,Country,Area,Activity,Sex,Age,Fatal (Y/N),Species
0,09,2016,Unprovoked,USA,Florida,Surfing,M,16,N,NaN
1,09,2016,Unprovoked,USA,Florida,Surfing,M,36,N,NaN
2,09,2016,Unprovoked,USA,Florida,Surfing,M,43,N,NaN
3,09,2016,Unprovoked,AUSTRALIA,Victoria,Surfing,M,NaN,N,NaN
4,09,2016,Unprovoked,AUSTRALIA,Victoria,Surfing,M,NaN,N,2 m shark
5,09,2016,Boat,AUSTRALIA,Western Australia,Fishing,NaN,NaN,N,NaN
6,09,2016,Unprovoked,USA,Florida,Wading,M,60s,N,3' to 4' shark
7,09,2016,Unprovoked,USA,Hawaii,Swimming,F,51,N,"Tiger shark, 10?"
8,09,2016,Unprovoked,NEW CALEDONIA,North Province,Kite surfing,M,50,Y,NaN
9,09,2016,Unprovoked,USA,South Carolina,Boogie boarding,F,12,N,NaN


In [114]:
data_clean.rename(columns={'Sex ':'Sex'}, inplace=True)

In [ ]:
data_clean['Sex'].value_counts()

In [115]:
data_clean["Sex"].fillna("Unknown", inplace = True)
display(data_clean)

,Month,Year,Type,Country,Area,Activity,Sex,Age,Fatal (Y/N),Species
0,09,2016,Unprovoked,USA,Florida,Surfing,M,16,N,NaN
1,09,2016,Unprovoked,USA,Florida,Surfing,M,36,N,NaN
2,09,2016,Unprovoked,USA,Florida,Surfing,M,43,N,NaN
3,09,2016,Unprovoked,AUSTRALIA,Victoria,Surfing,M,NaN,N,NaN
4,09,2016,Unprovoked,AUSTRALIA,Victoria,Surfing,M,NaN,N,2 m shark
5,09,2016,Boat,AUSTRALIA,Western Australia,Fishing,Unknown,NaN,N,NaN
6,09,2016,Unprovoked,USA,Florida,Wading,M,60s,N,3' to 4' shark
7,09,2016,Unprovoked,USA,Hawaii,Swimming,F,51,N,"Tiger shark, 10?"
8,09,2016,Unprovoked,NEW CALEDONIA,North Province,Kite surfing,M,50,Y,NaN
9,09,2016,Unprovoked,USA,South Carolina,Boogie boarding,F,12,N,NaN


In [88]:
data_clean['Sex'].value_counts()

M          2416
F           389
Unknown     230
Name: Sex, dtype: int64

In [116]:
data_clean = data_clean[(data_clean['Sex']=='M') | (data_clean['Sex']=='F') | (data_clean['Sex']=='Unknown')]
display(data_clean)

,Month,Year,Type,Country,Area,Activity,Sex,Age,Fatal (Y/N),Species
0,09,2016,Unprovoked,USA,Florida,Surfing,M,16,N,NaN
1,09,2016,Unprovoked,USA,Florida,Surfing,M,36,N,NaN
2,09,2016,Unprovoked,USA,Florida,Surfing,M,43,N,NaN
3,09,2016,Unprovoked,AUSTRALIA,Victoria,Surfing,M,NaN,N,NaN
4,09,2016,Unprovoked,AUSTRALIA,Victoria,Surfing,M,NaN,N,2 m shark
5,09,2016,Boat,AUSTRALIA,Western Australia,Fishing,Unknown,NaN,N,NaN
6,09,2016,Unprovoked,USA,Florida,Wading,M,60s,N,3' to 4' shark
7,09,2016,Unprovoked,USA,Hawaii,Swimming,F,51,N,"Tiger shark, 10?"
8,09,2016,Unprovoked,NEW CALEDONIA,North Province,Kite surfing,M,50,Y,NaN
9,09,2016,Unprovoked,USA,South Carolina,Boogie boarding,F,12,N,NaN


In [90]:
data_clean["Age"].fillna("Unknown", inplace = True)
display(data_clean)

,Month,Year,Type,Country,Area,Activity,Sex,Age,Fatal (Y/N),Species
0,09,2016,Unprovoked,USA,Florida,None,M,16,N,NaN
1,09,2016,Unprovoked,USA,Florida,None,M,36,N,NaN
2,09,2016,Unprovoked,USA,Florida,None,M,43,N,NaN
3,09,2016,Unprovoked,AUSTRALIA,Victoria,None,M,Unknown,N,NaN
4,09,2016,Unprovoked,AUSTRALIA,Victoria,None,M,Unknown,N,2 m shark
5,09,2016,Boat,AUSTRALIA,Western Australia,None,Unknown,Unknown,N,NaN
6,09,2016,Unprovoked,USA,Florida,None,M,60s,N,3' to 4' shark
7,09,2016,Unprovoked,USA,Hawaii,Swimming,F,51,N,"Tiger shark, 10?"
8,09,2016,Unprovoked,NEW CALEDONIA,North Province,None,M,50,Y,NaN
9,09,2016,Unprovoked,USA,South Carolina,None,F,12,N,NaN


In [137]:
data_clean["Age"].value_counts()

19                         95
17                         92
15                         90
20                         90
16                         88
18                         84
21                         75
22                         74
14                         72
25                         67
24                         67
29                         60
13                         58
26                         57
27                         50
28                         50
23                         49
32                         47
12                         45
30                         44
35                         42
10                         38
31                         38
40                         33
43                         32
34                         31
37                         29
36                         28
52                         27
42                         26
                           ..
37, 67, 35, 27,  ? & 27     1
                            1
6�        

In [139]:
def age(e):
    e=str(e)
    e = re.findall('\d+', e)
    if len(e)==1:
        return e[0]
    else:
        return'Unknown'

In [140]:
g = data['Age'].apply(age)
display(g)

0            16
1            36
2            43
3       Unknown
4       Unknown
5       Unknown
6            60
7            51
8            50
9            12
10      Unknown
11            9
12           22
13           25
14           37
16           49
18           21
19           22
22      Unknown
23           43
24           18
26           36
27      Unknown
28           31
29      Unknown
30            9
31           11
32      Unknown
35      Unknown
36           59
         ...   
3983    Unknown
3984         12
3996         19
3997         16
3999         70
4000         29
4001    Unknown
4004         27
4008         34
4009    Unknown
4010         13
4011    Unknown
4012         10
4013         23
4017    Unknown
4018    Unknown
4019         13
4020         38
4022         38
4023          7
4024    Unknown
4027    Unknown
4029    Unknown
4031         13
4032         26
4033         21
4034         15
4035          8
4036         18
4038         25
Name: Age, Length: 3047,

In [ ]:
data['Age'].value_counts()

In [126]:
def activity(u):
    u=str(u)
    if (re.search('[A-Z,a-z]wi+\w+',u )):
        return 'Swimming'
    elif (re.search('fish+\w+', u)):
        return 'Fishing'
    elif (re.search('[A-Z,a-z]ur+\w+',u)):
        return 'Surfing'
    elif (re.search('divi+\w+',u)):
        return 'Diving'
    elif (re.search('Bod+\w+', u)):
        return 'Surfing'
    else:
        return u

In [127]:
j = data_clean['Activity'].apply(activity)
display(j)

0                                     Surfing
1                                     Surfing
2                                     Surfing
3                                     Surfing
4                                     Surfing
5                                     Fishing
6                                      Wading
7                                    Swimming
8                                     Surfing
9                             Boogie boarding
10                                    Surfing
11                                    Surfing
12                                    Fishing
13                                    Surfing
14                                    Surfing
16                                     Wading
18                          SUP Foil boarding
19                                    Surfing
22                                   Kayaking
23                                 Lobstering
24                                    Surfing
26                                

In [129]:
data_clean['Activity'].value_counts()

Surfing                                                              786
Swimming                                                             359
Spearfishing                                                         192
Fishing                                                              163
Wading                                                               111
Standing                                                              67
Snorkeling                                                            54
Scuba diving                                                          43
Body boarding                                                         42
Diving                                                                40
Body surfing                                                          32
Kayaking                                                              26
Boogie boarding                                                       26
Free diving                                        

In [ ]:
order = data_clean[(data_clean["Injury"].isnull() == True)]
display(order)
order.shape #NO hay demasiados injury nulos así que de momento lo dejo pero los borraré

In [ ]:
fatal = data_clean[(data_clean["Fatal (Y/N)"].isnull() == True)]
display(fatal)
fatal.shape #NO hay demasiados fatal nulos así que de momento lo dejo pero los borrare

In [ ]:
time = data_clean[(data_clean["Time"].isnull() == True)]
display(time)
time.shape #NO hay demasiados fatal nulos así que de momento lo dejo pero los borrare



In [ ]:
data_clean['Case Number'].value_counts()